In [1]:
#TODO add STRUCT variables from dwarf info

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)
import posixpath
import sys,os,pickle
from elftools.elf.segments import Segment


from collections import defaultdict

import collections
import posixpath

import networkx as nx
from pyvis.network import Network
net = Network(notebook=True)
import matplotlib
import matplotlib.pyplot
from difflib import SequenceMatcher

import ntpath
from capstone import *
from capstone.x86 import *


ANALYSIS_DATA_PATH = '/home/nahid/temp_output/files/'
SRC_N_BIN_PATH = '/home/nahid/temp_dir/'

In [2]:



def get_bin_and_src_path(pkl_filename):
    proj_name = pkl_filename.split('_____')[0]
    bin_file_name = pkl_filename.split('_____')[1].split('.pkl')[0]
    bin_path = os.path.join( SRC_N_BIN_PATH ,proj_name , 'elfs', bin_file_name )
    src_dir_name  = [ dir_name  for dir_name in os.listdir( os.path.join(SRC_N_BIN_PATH ,proj_name)) if dir_name!='elfs'][0]
    src_path = os.path.join( SRC_N_BIN_PATH ,proj_name , src_dir_name )
    return bin_file_name ,proj_name, bin_path , src_path


def fix_src_path(cu_path):#TODO reduce global var usage
    compiler_machine_dependent_path = cu_path[:cu_path.find(project_name)]
    cu_path = cu_path.replace(compiler_machine_dependent_path , SRC_N_BIN_PATH)
    return cu_path

def check_dwarf_ok(filePath):
    with open(filePath, 'rb') as f:
        elffile = ELFFile( f )

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return False
        dwarfinfo = elffile.get_dwarf_info()
        
        try:
            if len(list(dwarfinfo.iter_CUs()))==0:
                return False
            for CU in dwarfinfo.iter_CUs():
                CU_DIR_PATH = None
                CU_FILENAME = None
                for attr in CU.get_top_DIE().attributes.values():
                    if attr.name == 'DW_AT_comp_dir':
                        CU_DIR_PATH = fix_src_path(attr.value.decode("utf-8"))
                    if attr.name == 'DW_AT_name':
                        CU_FILENAME = attr.value.decode("utf-8")
                line_program = dwarfinfo.line_program_for_CU(CU)
                if line_program is None:
                    print('  DWARF info is missing a line program for this CU')
                    return False
            return True
                
        except Exception as e:
            return False

def get_min_max_address(filePath):
    with open(filePath, 'rb') as f:
        elffile = ELFFile(f)

        dwarfinfo = elffile.get_dwarf_info()
        min_address = 10000000000000
        max_address = -100000000000
        for CU in dwarfinfo.iter_CUs():
            CU_DIR_PATH = None
            CU_FILENAME = None
            for attr in CU.get_top_DIE().attributes.values():
                if attr.name == 'DW_AT_comp_dir':
                    CU_DIR_PATH = fix_src_path(attr.value.decode("utf-8"))
                if attr.name == 'DW_AT_name':
                    CU_FILENAME = attr.value.decode("utf-8")

            line_program = dwarfinfo.line_program_for_CU(CU)


            cu_file_path  = os.path.join(CU_DIR_PATH, CU_FILENAME)

            for line_entry in line_program.get_entries():
                if line_entry.state!= None:
                    src_file_name = line_program.header['file_entry'][line_entry.state.file-1].name.decode("utf-8")
                    if src_file_name==CU_FILENAME: # no match means library C code
#                         if line_entry.state.line in bounds_matrix: #not always presend as disabled code might be present
                        if line_entry.state.address>max_address:
                            max_address = line_entry.state.address
            
                        if line_entry.state.address <min_address:
                            min_address = line_entry.state.address
    return min_address,max_address
         
def get_valid_instructions (filePath, addr_list, min_address, max_address):

    fh = open(filePath, 'rb')
    bin_bytearray = bytearray(fh.read())

    address_inst = {}
    
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    
    for addr in addr_list:
        

        ops = bin_bytearray[addr: ]

        #TODO make efficient
        for inst in md.disasm(ops, addr):
            if inst.address<=max_address and inst.address>=min_address:
                address_inst[inst.address] = inst
            break
            
    address_inst = collections.OrderedDict(sorted(address_inst.items()))
    return address_inst

In [3]:

analysed_pkl_filename = os.listdir(ANALYSIS_DATA_PATH)[3]
analysed_pkl_path = os.path.join(ANALYSIS_DATA_PATH  , analysed_pkl_filename )
pkl_name = ntpath.basename ( analysed_pkl_path )

binFileName, project_name, binary_path, src_dir_path = get_bin_and_src_path( pkl_name )
MIN_ADDRESS, MAX_ADDRESS =  get_min_max_address(binary_path)


with (open(analysed_pkl_path , "rb")) as openfile:
    bb_data , ins_data , tool_addresses_list = pickle.load(openfile)
            
VALID_INSTRUCTIONS_SET = get_valid_instructions(binary_path,tool_addresses_list,min_address=MIN_ADDRESS, max_address=MAX_ADDRESS)
len(tool_addresses_list),len(VALID_INSTRUCTIONS_SET)

In [5]:
min(list(VALID_INSTRUCTIONS_SET.keys())), max(list(VALID_INSTRUCTIONS_SET.keys()))

(13024, 22015)

In [6]:
VALID_INSTRUCTIONS_SET

OrderedDict([(13024, <CsInsn 0x32e0 [f30f1efa]: endbr64 >),
             (13028, <CsInsn 0x32e4 [836f0801]: sub dword ptr [rdi + 8], 1>),
             (13032, <CsInsn 0x32e8 [48830704]: add qword ptr [rdi], 4>),
             (13036, <CsInsn 0x32ec [83470c04]: add dword ptr [rdi + 0xc], 4>),
             (13040, <CsInsn 0x32f0 [c3]: ret >),
             (13056, <CsInsn 0x3300 [f30f1efa]: endbr64 >),
             (13060, <CsInsn 0x3304 [4863d6]: movsxd rdx, esi>),
             (13063, <CsInsn 0x3307 [48c1fe20]: sar rsi, 0x20>),
             (13067, <CsInsn 0x330b [48c1e202]: shl rdx, 2>),
             (13071, <CsInsn 0x330f [488d043a]: lea rax, [rdx + rdi]>),
             (13075, <CsInsn 0x3313 [01f2]: add edx, esi>),
             (13077, <CsInsn 0x3315 [89d2]: mov edx, edx>),
             (13079, <CsInsn 0x3317 [48c1e220]: shl rdx, 0x20>),
             (13083, <CsInsn 0x331b [c3]: ret >),
             (13088, <CsInsn 0x3320 [f30f1efa]: endbr64 >),
             (13092, <CsInsn 0x3324 [83

In [7]:
len(VALID_INSTRUCTIONS_SET)
    

1753

In [8]:

def line_entry_mapping(line_program,CU):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    if len(lp_entries)==0:
        return None
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state:# or lpe.state.file == 0
            continue
        filename = lpe_filename(line_program, lpe.state.file,CU)[0]
        filename_map[filename] += 1

    # for filename, lpe_count in filename_map.items():
    #     print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename(line_program, file_index, CU):
    
    
    die_dict = {}                    
    for attr in CU.get_top_DIE().attributes.values():
        die_dict[attr.name] = attr
    
    
    compilation_command = die_dict['DW_AT_producer'].value.decode("utf-8")
    
    if 'clang' in compilation_command.lower():
        COMPILER_SUBSTRACT = 1
    elif 'gnu' in compilation_command.lower():
        COMPILER_SUBSTRACT = 0
    lp_header = line_program.header
    file_entries = lp_header["file_entry"]
#     print(COMPILER_SUBSTRACT, compilation_command)
    
    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index -COMPILER_SUBSTRACT]
    dir_index = file_entry["dir_index"]

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode(),dir_index
    directory = lp_header["include_directory"][dir_index -COMPILER_SUBSTRACT]
    return posixpath.join(directory, file_entry.name).decode(),dir_index


def show_loclist(loclist, dwarfinfo, indent, cu_offset):
    """ Display a location list nicely, decoding the DWARF expressions
        contained within.
    """
    d = []
    for loc_entity in loclist:
        if isinstance(loc_entity, LocationEntry):
            d.append('%s <<%s>>' % (
                loc_entity,
                describe_DWARF_expr(loc_entity.loc_expr, dwarfinfo.structs, cu_offset)))
        else:
            d.append(str(loc_entity))
    return '\n'.join(indent + s for s in d)


########################################################
######################   DWARF PERSER #######################
###########################################################


def get_DIE_at_offset(CU, offset):
        for die in CU.iter_DIEs():
            if die.offset == CU.cu_offset+offset:
                return die 
        return None


##TODO FIX CONSTANT TYPE
def get_type_name(CU, offset):#get_DIE_at_offset(CU,attr.value)
    die = get_DIE_at_offset(CU, offset)
    
    if die.tag == 'DW_TAG_const_type':
        return "const"
    
    if die.tag == 'DW_TAG_pointer_type' :
        for _attr in die.attributes.values():
            if _attr.name== "DW_AT_type":
                
                return "*"+get_type_name(CU, _attr.value) 

    elif die.tag =='DW_TAG_subroutine_type':
        

        for _attr in die.attributes.values():
            if _attr.name== "DW_AT_sibling":
                return get_type_name(CU, _attr.value) 
            
            if _attr.name== "DW_AT_type":
                return "*"+get_type_name(CU, _attr.value) 

    for attr in die.attributes.values():
        if attr.name== "DW_AT_name":
            return attr.value.decode("utf-8")


    
    

    
############################################################
############################# CLANG #######################
###########################################################

# FUNCTION_DECL
# https://stackoverflow.com/questions/43460605/function-boundary-identification-using-libclang
# https://eli.thegreenplace.net/2011/07/03/parsing-c-in-python-with-clang


import clang.cindex
from clang.cindex import CursorKind


def get_all_var_types(source_path):
    srcFileName = source_path.split('/')[-1]
    idx = clang.cindex.Index.create()
    print(source_path)
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.VAR_DECL:
            # print(dir(f))
            print('file.name: ',f.extent.start.file.name)
            originFileName = f.extent.start.file.name.split('/')[-1]
            print('originFileName :',originFileName)
            print('displayname:  ',f.displayname)
            print('type.spelling:  ',f.type.spelling)
            print('f.extent.start.line: ', f.extent.start.line, "col: ",f.extent.start.column)
            print('\n')
            
            

   

def get_all_function_types(source_path):
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:
            # print(dir(f))
            print(f.displayname)
            print('function name: ',( f.spelling))
            print('Returns: ',(f.result_type.spelling))
            
            
            arg_len = len(list(f.type.argument_types()))
            if arg_len>0:
                arg_types = list(f.type.argument_types())
                for arg_type in arg_types:
                    print('arg_type:',arg_type.spelling)
                args = list(f.get_arguments())
                for arg in args:
                    print('arg:',arg.spelling)


            print("\n\n\n")

            
def get_function_boundaries(source_path): #TODO does not perse disabled Source code,not needed anyway
    
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    print("source_path: ",source_path)
    tu = idx.parse(source_path)

    for f in tu.cursor.walk_preorder():
        
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:

            function_name = f.displayname.split('(')[0]
            function_boundary_by_name[function_name]={}
            function_boundary_by_name[function_name] = { 'src_path':f.extent.start.file.name,
                              'src_file':f.extent.start.file.name.split('/')[-1],
                              'start_line':f.extent.start.line,
                              'start_col':f.extent.start.column,
                              'end_line':f.extent.end.line,
                              'end_col':f.extent.end.column}

    return function_boundary_by_name

def get_containing_function(source_file_path, line, col=0):
    function_boundary_by_name = get_function_boundaries(source_file_path)
    
    for function_name, item in function_boundary_by_name.items():
        if item['src_path'] == source_file_path:
            if line>= item['start_line'] and line<= item['end_line']:
                return function_name
        

def form_function_bound_metrix(src_bounds, src_file_name):
    bounds = {}
    for func_info in src_bounds.items():
        if func_info[1]['src_file'] == src_file_name:
            start_line  = func_info[1]['start_line']
            end_line    = func_info[1]['end_line']
            print(func_info[0] ,start_line , end_line)
            for i in range(start_line , end_line+1):
                bounds[i] = func_info[0]
               
    return bounds



def find_variables_per_line(source_path , line_to_function_matrix , dwarf_FUNC_PARAMS):
    print(line_to_function_matrix)
    srcFileName = source_path.split('/')[-1]
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    var_usage_matrix = {}
    for f in tu.cursor.walk_preorder():

        #TODO keep all with type info, explore CursorKind
        #TODO function ends  }  should relate with fucntion return type
        
        if f.kind in [CursorKind.PARM_DECL ,CursorKind.DECL_REF_EXPR, CursorKind.VAR_DECL]  :
            
            originFileName = f.extent.start.file.name.split('/')[-1]
            
            if srcFileName!=originFileName:
                continue


            line = f.extent.start.line
            col =f.extent.start.column
            type_info = f.type.spelling
            var_name = f.displayname

            if line not in var_usage_matrix:
                var_usage_matrix[line] = {}

            if line in line_to_function_matrix:# func declaration, global variables,  might not present
                if line_to_function_matrix[line] in dwarf_FUNC_PARAMS[source_path]:
                    #because wiredrly some function info are not in DWARF INFO
                    if var_name in dwarf_FUNC_PARAMS[source_path][line_to_function_matrix[line]]:
                        var_usage_matrix[line][col] = {
                                        'name'       : f.displayname ,
                                        'dwarf_info' : dwarf_FUNC_PARAMS[source_path][line_to_function_matrix[line]][var_name],
                                        'type'       : f.type.spelling }
    return var_usage_matrix

            
###################################################################
########  Find Src Code by filepath, line and col no  #############
##################################################################

def getSource(sourceFilePath, row , col):
    print(sourceFilePath, row , col)
    sourceFile = open(sourceFilePath, "r")
    fileContent = sourceFile.readlines()
    row_content =  fileContent[row-1]
    row_content = row_content[:(col-1)] + '|'+row_content[(col-1)]+'|' +row_content[col:]
    
    return row_content



In [9]:
# dir(clang.cindex.CursorKind)

In [10]:

##############################################################################
#################   RELATE FUNCTIONS TO ADDRESSES #############################
###############################################################################

import collections
            

lineinfo_address_subprogram = {}
with open(binary_path, 'rb') as f:
    elffile = ELFFile(f)

    if not elffile.has_dwarf_info():
        print('  file has no DWARF info')
        exit(0)

    dwarfinfo = elffile.get_dwarf_info()
    for CU in dwarfinfo.iter_CUs():
        CU_DIR_PATH = None
        CU_FILENAME = None
        for attr in CU.get_top_DIE().attributes.values():
            if attr.name == 'DW_AT_comp_dir':
                CU_DIR_PATH = fix_src_path(attr.value.decode("utf-8"))
            if attr.name == 'DW_AT_name':
                CU_FILENAME = attr.value.decode("utf-8")
        
        print('  Found a compile unit at offset %s, length %s' % (
            CU.cu_offset, CU['unit_length']))

        # Every compilation unit in the DWARF information may or may not
        # have a corresponding line program in .debug_line.
        line_program = dwarfinfo.line_program_for_CU(CU)
        if line_program is None:
            print('  DWARF info is missing a line program for this CU')
            continue
        

        cu_file_path  = os.path.join(CU_DIR_PATH, CU_FILENAME)

        bounds_matrix = form_function_bound_metrix( get_function_boundaries(cu_file_path)  , CU_FILENAME)
        

        for line_entry in line_program.get_entries():
            if line_entry.state!= None:
                src_file_name = line_program.header['file_entry'][line_entry.state.file-1].name.decode("utf-8")
                if src_file_name==CU_FILENAME: # no match means library C code
                    
                    if line_entry.state.line in bounds_matrix: #not always presend as disabled code might be present
                        lineinfo_address_subprogram[line_entry.state.address]  =   {
                            'func':bounds_matrix[line_entry.state.line], 
                            'srcPath':cu_file_path,
                            'lineinfo':line_entry.state
                        } 


#TODO make efficient with valid address only

lineinfo_address_subprogram = collections.OrderedDict(sorted(lineinfo_address_subprogram.items()))
lineinfo_address_subprogram_complete = {}

min_address = min(lineinfo_address_subprogram.keys())
max_address = max(lineinfo_address_subprogram.keys())



print('DBG: MIN MAX',min_address ,max_address)
temp_subprogram = lineinfo_address_subprogram[min_address]
for i in range(min_address,max_address+1):
    if i in lineinfo_address_subprogram:
        temp_subprogram = lineinfo_address_subprogram[i]
    lineinfo_address_subprogram_complete[i] = temp_subprogram
# lineinfo_address_subprogram_complete




  Found a compile unit at offset 0, length 22459
source_path:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c
xcb_xv_port_next 22 28
xcb_xv_port_end 30 38
xcb_xv_encoding_next 40 46
xcb_xv_encoding_end 48 56
xcb_xv_rational_next 58 64
xcb_xv_rational_end 66 74
xcb_xv_format_next 76 82
xcb_xv_format_end 84 92
xcb_xv_adaptor_info_sizeof 94 144
xcb_xv_adaptor_info_name 146 150
xcb_xv_adaptor_info_name_length 152 156
xcb_xv_adaptor_info_name_end 158 166
xcb_xv_adaptor_info_formats 168 173
xcb_xv_adaptor_info_formats_length 175 179
xcb_xv_adaptor_info_formats_iterator 181 190
xcb_xv_adaptor_info_next 192 201
xcb_xv_adaptor_info_end 203 213
xcb_xv_encoding_info_sizeof 215 253
xcb_xv_encoding_info_name 255 259
xcb_xv_encoding_info_name_length 261 265
xcb_xv_encoding_info_name_end 267 275
xcb_xv_encoding_info_next 277 286
xcb_xv_encoding_info_end 288 298
xcb_xv_image_sizeof 300 353
xcb_xv_image_pitches 355 359
xcb_xv_image_pitches_length 361 365
xcb_xv_image_pitches_end 367 

In [11]:
# (12832, 22015) , err-> 12832
# lineinfo_address_subprogram_complete[12833]

In [12]:
# https://github.com/eliben/pyelftools/blob/master/examples/dwarf_location_info.py

from elftools.dwarf.locationlists import LocationParser, LocationExpr
FUNC_PARAMS = {}

def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        # get_dwarf_info returns a DWARFInfo context object, which is the
        # starting point for all DWARF-based processing in pyelftools.
        dwarfinfo = elffile.get_dwarf_info()
        # The location lists are extracted by DWARFInfo from the .debug_loc
        # section, and returned here as a LocationLists object.
        location_lists = dwarfinfo.location_lists()
        

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        # Create a LocationParser object that parses the DIE attributes and
        # creates objects representing the actual location information.
        loc_parser = LocationParser(location_lists)
        section_offset = dwarfinfo.debug_info_sec.global_offset
        # Offset of the .debug_info section in the stream
        
        
        for CU in dwarfinfo.iter_CUs():
            CU_DIR_PATH = None
            CU_FILENAME = None
            for attr in CU.get_top_DIE().attributes.values():
                if attr.name == 'DW_AT_comp_dir':
                    CU_DIR_PATH = fix_src_path(attr.value.decode("utf-8"))
                if attr.name == 'DW_AT_name'    :
                    CU_FILENAME = attr.value.decode("utf-8")

            line_program = dwarfinfo.line_program_for_CU(CU)

         
            CU_dictionary_key = os.path.join(CU_DIR_PATH, CU_FILENAME)
            if CU_dictionary_key not in FUNC_PARAMS:
                FUNC_PARAMS[CU_dictionary_key] = {}
            
            print('  Found a compile unit at offset %s, length %s' % (
                CU.cu_offset, CU['unit_length']))

            # A CU provides a simple API to iterate over all the DIEs in it.
            die_depth = 0
            are_DIEs_of_function = False
            FUNC_name = None
            for DIE in CU.iter_DIEs():
                
                ############################################################
                #############   Prasing Function DIEs start ################

                
                if DIE.tag == 'DW_TAG_subprogram':
                    if 'DW_AT_low_pc' in DIE.attributes and 'DW_AT_high_pc' in DIE.attributes :
                        low_pc = DIE.attributes['DW_AT_low_pc'].value
                        high_pc = DIE.attributes['DW_AT_high_pc'].value
                        
                        print("Low PC: ",hex(low_pc) , " High PC" , hex(high_pc))
                    else:
                        print("NO PC given")
                    are_DIEs_of_function = True
                    
                    for attr in DIE.attributes.values():
                        if attr.name == "DW_AT_name": #FUNC NAME
                            FUNC_name = attr.value.decode("utf-8")
                            if FUNC_name not in FUNC_PARAMS[CU_dictionary_key]:
                                FUNC_PARAMS[CU_dictionary_key][FUNC_name] ={}
                            print("SUBPROGRAM: ",FUNC_name)
                            
                if DIE.tag in[ 'DW_TAG_formal_parameter','DW_TAG_variable' ,'DW_TAG_member']:
                    tags = [attr.name for attr in DIE.attributes.values()]
                    PARAM_name = None
                    if FUNC_name==None:
                        
                        FUNC_name ="global"
                        
                        if FUNC_name not in FUNC_PARAMS[CU_dictionary_key]:
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name]={}
                        
                    if "DW_AT_name" in tags:
                        
                        die_dict = {}
                        
                        for attr in DIE.attributes.values():
                            die_dict[attr.name] = attr
                        
                        PARAM_name = die_dict['DW_AT_name'].value.decode("utf-8")
                        
                        if PARAM_name not in FUNC_PARAMS[CU_dictionary_key][FUNC_name]:
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {}
                        var_type = DIE.tag.split('_')[-1]
                        FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {'type':get_type_name(CU,die_dict['DW_AT_type'].value) , 'kind':var_type}
                        
#                         print(die_dict)
                        # Check if this attribute contains location information
#                         if loc_parser.attribute_has_location(die_dict['DW_AT_location'], CU['version']):
                        if 'DW_AT_location' in die_dict:

                            try:
                                loc = loc_parser.parse_from_attribute(die_dict['DW_AT_location'],
                                                                      CU['version'])
                                
#                                 print(CU_dictionary_key,FUNC_name,PARAM_name)
                                if isinstance(loc, LocationExpr):
                                    loc_info_str = describe_DWARF_expr(loc.loc_expr, dwarfinfo.structs, CU.cu_offset)
                                    offset_temp = (loc_info_str.split('-')[-1]).split(')')[0]
#                                     print('1a ',loc_info_str, offset_temp)
#                                     print('1b ', PARAM_name,loc_info_str, int(offset_temp)-LOCATION_SUBSTRACT_FACTOR)
                                    FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= loc_info_str

                                elif isinstance(loc, list):
                                    print(PARAM_name,show_loclist(loc,dwarfinfo,'      ', CU.cu_offset))
                                    FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= show_loclist(loc,
                                                       dwarfinfo,'      ', CU.cu_offset)
                            except:

                                print("ERROR",DIE)
                                pass

                ###############################################
                #############  parsing  Function DIEs ends ################
                


                
                if DIE.is_null(): #https://chromium.googlesource.com/chromiumos/third_party/pyelftools/+/25a77f7738d7fe824f2ed4d33a123136b9d8e88a/scripts/readelf.py
                    are_DIEs_of_function = False
                    FUNC_name = None
                    
                    die_depth -= 1
                    continue
                if DIE.has_children:
                    die_depth += 1
                    

process_file(binary_path)





def create_variable_per_line_matrix(filename ,FUNCTION_PARAMS):
    variables_in_line_matrix_all_files = {}
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return
        dwarfinfo = elffile.get_dwarf_info()

        location_lists = dwarfinfo.location_lists()
        

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        loc_parser = LocationParser(location_lists)
        section_offset = dwarfinfo.debug_info_sec.global_offset
        # Offset of the .debug_info section in the stream
        
        
        for CU in dwarfinfo.iter_CUs():
            CU_DIR_PATH = None
            CU_FILENAME = None
            for attr in CU.get_top_DIE().attributes.values():
                if attr.name == 'DW_AT_comp_dir':
                    CU_DIR_PATH = fix_src_path(attr.value.decode("utf-8"))
                if attr.name == 'DW_AT_name':
                    CU_FILENAME = attr.value.decode("utf-8")

            #########
            cu_src_path = os.path.join(CU_DIR_PATH, CU_FILENAME)
            cu_func_boundaries =get_function_boundaries(cu_src_path )
            cu_src_line_to_function_matrix = form_function_bound_metrix(cu_func_boundaries , CU_FILENAME)
            variables_in_line_matrix = find_variables_per_line(cu_src_path, cu_src_line_to_function_matrix , FUNCTION_PARAMS)
            variables_in_line_matrix_all_files[cu_src_path] = variables_in_line_matrix
            #########
    return variables_in_line_matrix_all_files
variables_in_line = create_variable_per_line_matrix(binary_path, FUNC_PARAMS)

Processing file: /home/nahid/temp_dir/libxcb-shm0:amd64/elfs/libxcb-xv.so.0.0.0
  Found a compile unit at offset 0, length 22459
NO PC given
SUBPROGRAM:  xcb_wait_for_reply
NO PC given
SUBPROGRAM:  xcb_send_request
Low PC:  0x54e0  High PC 0x124
SUBPROGRAM:  xcb_xv_shm_put_image
c       LocationEntry(entry_offset=4, entry_length=19, begin_offset=21728, end_offset=21987, loc_expr=[85], is_absolute=False) <<(DW_OP_reg5 (rdi))>>
      LocationEntry(entry_offset=23, entry_length=22, begin_offset=21987, end_offset=22020, loc_expr=[243, 1, 85, 159], is_absolute=False) <<(DW_OP_GNU_entry_value: (DW_OP_reg5 (rdi)); DW_OP_stack_value)>>
port       LocationEntry(entry_offset=67, entry_length=19, begin_offset=21728, end_offset=21777, loc_expr=[84], is_absolute=False) <<(DW_OP_reg4 (rsi))>>
      LocationEntry(entry_offset=86, entry_length=21, begin_offset=21777, end_offset=21987, loc_expr=[145, 228, 126], is_absolute=False) <<(DW_OP_fbreg: -156)>>
      LocationEntry(entry_offset=107, entry_lengt

Low PC:  0x4ab0  High PC 0x74
SUBPROGRAM:  xcb_xv_set_port_attribute
c       LocationEntry(entry_offset=7844, entry_length=19, begin_offset=19120, end_offset=19209, loc_expr=[85], is_absolute=False) <<(DW_OP_reg5 (rdi))>>
      LocationEntry(entry_offset=7863, entry_length=22, begin_offset=19209, end_offset=19236, loc_expr=[243, 1, 85, 159], is_absolute=False) <<(DW_OP_GNU_entry_value: (DW_OP_reg5 (rdi)); DW_OP_stack_value)>>
port       LocationEntry(entry_offset=7907, entry_length=19, begin_offset=19120, end_offset=19153, loc_expr=[84], is_absolute=False) <<(DW_OP_reg4 (rsi))>>
      LocationEntry(entry_offset=7926, entry_length=21, begin_offset=19153, end_offset=19209, loc_expr=[145, 148, 127], is_absolute=False) <<(DW_OP_fbreg: -108)>>
      LocationEntry(entry_offset=7947, entry_length=22, begin_offset=19209, end_offset=19236, loc_expr=[243, 1, 84, 159], is_absolute=False) <<(DW_OP_GNU_entry_value: (DW_OP_reg4 (rsi)); DW_OP_stack_value)>>
attribute       LocationEntry(entry_offset=

c       LocationEntry(entry_offset=11423, entry_length=19, begin_offset=17984, end_offset=18071, loc_expr=[85], is_absolute=False) <<(DW_OP_reg5 (rdi))>>
      LocationEntry(entry_offset=11442, entry_length=22, begin_offset=18071, end_offset=18098, loc_expr=[243, 1, 85, 159], is_absolute=False) <<(DW_OP_GNU_entry_value: (DW_OP_reg5 (rdi)); DW_OP_stack_value)>>
port       LocationEntry(entry_offset=11486, entry_length=19, begin_offset=17984, end_offset=18026, loc_expr=[84], is_absolute=False) <<(DW_OP_reg4 (rsi))>>
      LocationEntry(entry_offset=11505, entry_length=21, begin_offset=18026, end_offset=18071, loc_expr=[145, 152, 127], is_absolute=False) <<(DW_OP_fbreg: -104)>>
      LocationEntry(entry_offset=11526, entry_length=22, begin_offset=18071, end_offset=18098, loc_expr=[243, 1, 84, 159], is_absolute=False) <<(DW_OP_GNU_entry_value: (DW_OP_reg4 (rsi)); DW_OP_stack_value)>>
drawable       LocationEntry(entry_offset=11570, entry_length=19, begin_offset=17984, end_offset=18035, loc

i       LocationEntry(entry_offset=20610, entry_length=28, begin_offset=14363, end_offset=14363, loc_expr=[117, 0, 113, 0, 34, 159, 147, 8, 147, 8], is_absolute=False) <<(DW_OP_breg5 (rdi): 0; DW_OP_breg1 (rdx): 0; DW_OP_plus; DW_OP_stack_value; DW_OP_piece: 8; DW_OP_piece: 8)>>
      LocationEntry(entry_offset=20638, entry_length=32, begin_offset=14363, end_offset=14363, loc_expr=[117, 0, 113, 0, 34, 159, 147, 8, 48, 159, 147, 4, 147, 4], is_absolute=False) <<(DW_OP_breg5 (rdi): 0; DW_OP_breg1 (rdx): 0; DW_OP_plus; DW_OP_stack_value; DW_OP_piece: 8; DW_OP_lit0; DW_OP_stack_value; DW_OP_piece: 4; DW_OP_piece: 4)>>
      LocationEntry(entry_offset=20670, entry_length=33, begin_offset=14363, end_offset=14367, loc_expr=[117, 0, 113, 0, 34, 159, 147, 8, 48, 159, 147, 4, 81, 147, 4], is_absolute=False) <<(DW_OP_breg5 (rdi): 0; DW_OP_breg1 (rdx): 0; DW_OP_plus; DW_OP_stack_value; DW_OP_piece: 8; DW_OP_lit0; DW_OP_stack_value; DW_OP_piece: 4; DW_OP_reg1 (rdx); DW_OP_piece: 4)>>
      Location

xcb_align_to       LocationEntry(entry_offset=22961, entry_length=20, begin_offset=13876, end_offset=13876, loc_expr=[48, 159], is_absolute=False) <<(DW_OP_lit0; DW_OP_stack_value)>>
      LocationEntry(entry_offset=22981, entry_length=20, begin_offset=13876, end_offset=13876, loc_expr=[52, 159], is_absolute=False) <<(DW_OP_lit4; DW_OP_stack_value)>>
      LocationEntry(entry_offset=23001, entry_length=20, begin_offset=13876, end_offset=13887, loc_expr=[49, 159], is_absolute=False) <<(DW_OP_lit1; DW_OP_stack_value)>>
Low PC:  0x35e0  High PC 0x47
SUBPROGRAM:  xcb_xv_encoding_info_end
i       LocationEntry(entry_offset=23043, entry_length=24, begin_offset=13792, end_offset=13805, loc_expr=[85, 147, 8, 84, 147, 8], is_absolute=False) <<(DW_OP_reg5 (rdi); DW_OP_piece: 8; DW_OP_reg4 (rsi); DW_OP_piece: 8)>>
      LocationEntry(entry_offset=23067, entry_length=25, begin_offset=13805, end_offset=13840, loc_expr=[145, 96, 147, 8, 84, 147, 8], is_absolute=False) <<(DW_OP_fbreg: -32; DW_OP_piec

xcb_xv_port_next 22 28
xcb_xv_port_end 30 38
xcb_xv_encoding_next 40 46
xcb_xv_encoding_end 48 56
xcb_xv_rational_next 58 64
xcb_xv_rational_end 66 74
xcb_xv_format_next 76 82
xcb_xv_format_end 84 92
xcb_xv_adaptor_info_sizeof 94 144
xcb_xv_adaptor_info_name 146 150
xcb_xv_adaptor_info_name_length 152 156
xcb_xv_adaptor_info_name_end 158 166
xcb_xv_adaptor_info_formats 168 173
xcb_xv_adaptor_info_formats_length 175 179
xcb_xv_adaptor_info_formats_iterator 181 190
xcb_xv_adaptor_info_next 192 201
xcb_xv_adaptor_info_end 203 213
xcb_xv_encoding_info_sizeof 215 253
xcb_xv_encoding_info_name 255 259
xcb_xv_encoding_info_name_length 261 265
xcb_xv_encoding_info_name_end 267 275
xcb_xv_encoding_info_next 277 286
xcb_xv_encoding_info_end 288 298
xcb_xv_image_sizeof 300 353
xcb_xv_image_pitches 355 359
xcb_xv_image_pitches_length 361 365
xcb_xv_image_pitches_end 367 375
xcb_xv_image_offsets 377 382
xcb_xv_image_offsets_length 384 388
xcb_xv_image_offsets_end 390 399
xcb_xv_image_data 401 406
x

In [13]:
# FUNC_PARAMS#['/home/nahid/pkg_data/apg/apg-2.2.3.dfsg.1/rnd.c']

In [14]:
# source_file = '/home/nahid/reverse/binaries/c_many/stack.c'

# variable_matrix

In [15]:

def diff_dict(matrix):
    for i in range (len(matrix.keys()) -1):
            ith_key = [*matrix.keys()][i]
            i_plus_1th_key = [*matrix.keys()][i+1]
            matrix[ith_key] = matrix[ith_key] -matrix[i_plus_1th_key] 
    return matrix

def vars_to_types(var_list, cu_path, func):
    types=[]
    for variable_name in var_list:
        types.append(FUNC_PARAMS[cu_path][func][variable_name]['type'])
    return types


#todo try func to address 
def build_line_to_relatedAddresses_matrix(address_lineinfo):#lineinfo_address_subprogram_complete
    line_address = {}
    for address  in VALID_INSTRUCTIONS_SET:#address_lineinfo.items():
        info = address_lineinfo[address]
        line          = info['lineinfo'].line
        col           = info['lineinfo'].column
        src_filepath  = info['srcPath']
        func          = info['func']
        
        key= str(line)+"_"+ str(col)
        
        if src_filepath not in line_address:
            line_address[src_filepath] ={}
        if func not in line_address[src_filepath]:
            line_address[src_filepath][func] = {}
        if key not in line_address[src_filepath][func]:
            line_address[src_filepath][func][key] = []
        
        line_address[src_filepath][func][key].append(address)
        
    return line_address

def assign_twin_instructions_types(addr_list, type_list, twin_dict):
        inst_to_type_dict = dict(zip(addr_list, type_list))

        for main_address, twin_list in twin_dict.items():
            for twin in twin_list:
                if main_address in inst_to_type_dict:
                    inst_to_type_dict[twin] = inst_to_type_dict[main_address]
        return inst_to_type_dict
         
# def get    
    
line_to_address_matrix = build_line_to_relatedAddresses_matrix(lineinfo_address_subprogram_complete)

# print(line_to_address_matrix)

for cu_path, all_func_data in line_to_address_matrix.items():
    for func, func_data in all_func_data.items():
        for line_col, line_addresses in func_data.items():
            line = int(line_col.split('_')[0])
            print('\n\n\nfile: ',cu_path,' func:',func,'\nline: ',line,'\n\n')
            
            #################### PROCESS ADDRESS LIST ##############################
            
            inst_matrix = {  }
            twin_instructions = {}
            for address in line_addresses:
                address_hex = hex(address)
                inst = VALID_INSTRUCTIONS_SET[address]
                instrctionCode = (address_hex+":\t"+ inst.mnemonic+" "+inst.op_str).ljust(45)

                disp = None
                if len(inst.operands) > 0 :
                    oc=-1
                    for o in inst.operands:
                        oc += 1
                        if o.type == CS_OP_MEM:
                            if o.value.mem.disp != 0:
                                disp = o.value.mem.disp
                                
                                if disp not in inst_matrix.values():
                                    inst_matrix[address_hex]=disp
                                else:
                                    
                                    twin_hex = list(inst_matrix.keys())[list(inst_matrix.values()).index(disp)] 
                                    if twin_hex not in twin_instructions:
                                        twin_instructions[twin_hex] = [address_hex]
                                    else:
                                        twin_instructions[twin_hex].append(address_hex)
                                    #TODO twin inst
            inst_matrix = dict(sorted(inst_matrix.items(), key=lambda x: x[1] , reverse=True))
            print(inst_matrix.values())
            

            #######################  PROCESS SRC VARIABLES #############################
            if line in variables_in_line[cu_path]: #ALL LINES SHOULD BE VALID, should not check
                var_list = variables_in_line[cu_path][line]
                print("DBG: ",cu_path ,line, var_list)
                var_matrix = {}               
                for col,var in var_list.items():

                    if 'location' in var['dwarf_info'] :
                        if len(var['dwarf_info']['location'])<15: #TODO, use regex.
                            var_matrix[var['name']] = int(var['dwarf_info']['location'].split(':')[-1][:-1])
                
                var_matrix = dict (sorted(var_matrix.items(), key=lambda x: x[1] , reverse=True))
                
                print(var_matrix.values())
                
                
                
                ########################################
                ############# Compare & Align  ################
                ########################################
                inst_matrix_len = len(inst_matrix.items())
                var_matrix_len  = len( var_matrix.items())
                
                #TODO
                # rule 1: they have single inst and single var, so just match
                if inst_matrix_len==1 and var_matrix_len==1:
                    
                    types = vars_to_types(list(var_matrix.keys()), cu_path, func)
                    insts = list(inst_matrix.keys())
                    
                    inst_to_type = assign_twin_instructions_types(insts,types,twin_instructions)
                    print('Single > ', inst_to_type )
                
                #TODO
                # rule 2: if one have 1 item and another have 1+ item, can match
                #         only with coloumn alignment
                if 1 in [inst_matrix_len,var_matrix_len] and \
                        abs(inst_matrix_len-var_matrix_len)>0:
                    continue
                
                #TODO
                # rule 3: if there are multiple longest matches
                pass
                
                
                
                
                inst_matrix = diff_dict(inst_matrix)

                var_matrix = diff_dict(var_matrix)
                
            
                match = SequenceMatcher(None, 
                                        list(var_matrix.values()), 
                                        list(inst_matrix.values())).find_longest_match()
                
                if match.size>0:#found matching seq
                    print("MATCHED!",match,var_matrix.keys() ,inst_matrix.keys() )

                    var_matches  = list(var_matrix.keys()) [match.a:(match.a+match.size)+1]
                    inst_matches = list(inst_matrix.keys())[match.b:(match.b+match.size)+1]
                    
                    print('var_matches: ',var_matches , ' INST matches:',inst_matches)

                    ### assign types
                    types = vars_to_types(var_matches, cu_path, func)
                    print('inst_matches: ', inst_matches, '\n types', types )
                    
                    ### handle twin,
                    inst_to_type = assign_twin_instructions_types(inst_matches,types,twin_instructions)
                    print(inst_to_type)
                        
                    
                ########################
#                 break
#         break
        
#     break
# line_to_address_matrix
# var_list





file:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c  func: xcb_xv_port_next 
line:  24 


dict_values([])



file:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c  func: xcb_xv_port_next 
line:  25 


dict_values([8])
DBG:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 25 {7: {'name': 'i', 'dwarf_info': {'type': '*xcb_xv_port_iterator_t', 'kind': 'parameter', 'location': '(DW_OP_reg5 (rdi))'}, 'type': 'xcb_xv_port_iterator_t *'}}
dict_values([])



file:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c  func: xcb_xv_port_next 
line:  26 


dict_values([])
DBG:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 26 {7: {'name': 'i', 'dwarf_info': {'type': '*xcb_xv_port_iterator_t', 'kind': 'parameter', 'location': '(DW_OP_reg5 (rdi))'}, 'type': 'xcb_xv_port_iterator_t *'}}
dict_values([])



file:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c  func: xcb_xv_port_next 
line

dict_values([19499, 64])



file:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c  func: xcb_xv_put_video 
line:  1023 


dict_values([8])
DBG:  /home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 1023 {5: {'name': 'xcb_out', 'dwarf_info': {'type': 'xcb_xv_put_video_request_t', 'kind': 'variable', 'location': '(DW_OP_fbreg: -128)'}, 'type': 'xcb_xv_put_video_request_t'}, 24: {'name': 'drawable', 'dwarf_info': {'type': 'xcb_drawable_t', 'kind': 'parameter', 'location': '      LocationEntry(entry_offset=14950, entry_length=19, begin_offset=16400, end_offset=16446, loc_expr=[81], is_absolute=False) <<(DW_OP_reg1 (rdx))>>\n      LocationEntry(entry_offset=14969, entry_length=21, begin_offset=16446, end_offset=16573, loc_expr=[145, 136, 127], is_absolute=False) <<(DW_OP_fbreg: -120)>>\n      LocationEntry(entry_offset=14990, entry_length=22, begin_offset=16573, end_offset=16600, loc_expr=[243, 1, 81, 159], is_absolute=False) <<(DW_OP_GNU_entry_value: (DW_OP_

In [16]:
lineinfo_address_subprogram_complete

{13024: {'func': 'xcb_xv_port_next',
  'srcPath': '/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c',
  'lineinfo': <LineState 7f99bab9a940:
    address = 0x32e0
    file = 1
    line = 24
    column = 1
    is_stmt = False
    basic_block = False
    end_sequence = False
    prologue_end = False
    epilogue_begin = False
    isa = 0
    discriminator = 0>},
 13025: {'func': 'xcb_xv_port_next',
  'srcPath': '/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c',
  'lineinfo': <LineState 7f99bab9a940:
    address = 0x32e0
    file = 1
    line = 24
    column = 1
    is_stmt = False
    basic_block = False
    end_sequence = False
    prologue_end = False
    epilogue_begin = False
    isa = 0
    discriminator = 0>},
 13026: {'func': 'xcb_xv_port_next',
  'srcPath': '/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c',
  'lineinfo': <LineState 7f99bab9a940:
    address = 0x32e0
    file = 1
    line = 24
    column = 1
    is_stmt = False
 

In [17]:

######################################3#########
########### ILLUSTRATE in file ##################
#################################################


REGISTER_SUBSTRACT_FACTOR = -0
dir_path = './../../binaries/c_many/'
with open(binFileName+'.s', 'w') as outFile:
    # outFile.write('file contents\n')
    lastSource = ""
    for address in VALID_INSTRUCTIONS_SET:
        address_hex = hex(address)
        print(address_hex)
        inst = VALID_INSTRUCTIONS_SET[address]
        instrctionCode = (address_hex+":\t"+ inst.mnemonic+" "+inst.op_str).ljust(45)

        OFFSET = None
        if len(inst.operands) > 0 :
            c=-1
            for o in inst.operands:
                c += 1
                if o.type == CS_OP_MEM:
                    print("\t\toperands[%u].type: MEM" %c)
                    if o.value.mem.base != 0:
                        print("\t\t\toperands[%u].mem.base: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.base)))
                    if o.value.mem.index != 0:
                        print("\t\t\toperands[%u].mem.index: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.index)))
                    if o.value.mem.disp != 0:
                        print("\t\t\toperands[%u].mem.disp: 0x%x" \
                            %(c, o.value.mem.disp))
                        OFFSET = o.value.mem.disp
                    print(hex(o.value.mem.disp),o.value.mem.disp)
                    

        if address in lineinfo_address_subprogram_complete:
            if lineinfo_address_subprogram_complete[address]['lineinfo'].address == address: 
                srcFilePath = lineinfo_address_subprogram_complete[address]['srcPath']
                if srcFilePath!=lastSource:
                    outFile.write("\n"+ '#'*100+"\n"+ srcFilePath.rjust(45) +'\n'+'#'*100+ "\n\n")
                    lastSource = srcFilePath


                src_line_no  = lineinfo_address_subprogram_complete[address]['lineinfo'].line
                src_col_no   = lineinfo_address_subprogram_complete[address]['lineinfo'].column
                sourceCode = getSource(srcFilePath,src_line_no, src_col_no)
                function_name = lineinfo_address_subprogram_complete[address]['func']
                
                print('##############',sourceCode)

                if '\n' not in  sourceCode:
                    sourceCode+=sourceCode+"\n"
                outFile.write(instrctionCode+"#"+ sourceCode  )
                print(instrctionCode+"#"+ sourceCode)



            else:

                outFile.write(instrctionCode+ '\n'  )
                print(instrctionCode)
            if OFFSET:
                outFile.write("MEMORY OFFSET:     "+str(hex(OFFSET))+"     "+str(OFFSET)+ "  >>"+str(OFFSET-REGISTER_SUBSTRACT_FACTOR)+'\n\n')
                pass

0x32e0
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 24 1
############## |{|

0x32e0:	endbr64                              #|{|

0x32e4
		operands[0].type: MEM
			operands[0].mem.base: REG = rdi
			operands[0].mem.disp: 0x8
0x8 8
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 25 5
##############     |-|-i->rem;

0x32e4:	sub dword ptr [rdi + 8], 1           #    |-|-i->rem;

0x32e8
		operands[0].type: MEM
			operands[0].mem.base: REG = rdi
0x0 0
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 26 5
##############     |+|+i->data;

0x32e8:	add qword ptr [rdi], 4               #    |+|+i->data;

0x32ec
		operands[0].type: MEM
			operands[0].mem.base: REG = rdi
			operands[0].mem.disp: 0xc
0xc 12
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 27 14
##############     i->index |+|= sizeof(xcb_xv_port_t);

0x32ec:	add dword ptr [rdi + 0xc], 4         #    i->index |+|= sizeof(xcb_xv_port_t);

0x32f0
/home/nahid/temp_d

##############     i.data = ((char *) (R + 1)) + (R|-|>name_size);

0x3584:	movzx edx, word ptr [rdi + 4]        #    i.data = ((char *) (R + 1)) + (R|-|>name_size);

0x3588
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 271 33
##############     i.data = ((char *) (R + 1)) |+| (R->name_size);

0x3588:	add rdx, 0x14                        #    i.data = ((char *) (R + 1)) |+| (R->name_size);

0x358c
		operands[1].type: MEM
			operands[1].mem.base: REG = rdi
			operands[1].mem.index: REG = rdx
0x0 0
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 271 33
##############     i.data = ((char *) (R + 1)) |+| (R->name_size);

0x358c:	lea rax, [rdi + rdx]                 #    i.data = ((char *) (R + 1)) |+| (R->name_size);

0x3590
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 274 12
##############     return |i|;

0x3590:	shl rdx, 0x20                        #    return |i|;

0x3594
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build

##############     i->index |=| (char *) child.data - (char *) i->data;

0x385a:	mov dword ptr [rbx + 0xc], eax       #    i->index |=| (char *) child.data - (char *) i->data;

0x385d
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 519 1
############## |}|

0x385d:	add rsp, 8                           #|}|

0x3861
0x3861:	pop rbx                              
0x3862
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 519 1
############## |}|

0x3862:	pop rbp                              #|}|

0x3863
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 519 1
############## |}|

0x3863:	ret                                  #|}|

0x3870
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 523 1
############## |{|

0x3870:	endbr64                              #|{|

0x3874
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 523 1
############## |{|

0x3874:	push rbx                             #|{|

0x3875
0x3875:	sub r

##############     xcb_parts[2].iov_base |=| (char *) &xcb_out;

0x4b53:	lea rax, [rsp + 4]                   #    xcb_parts[2].iov_base |=| (char *) &xcb_out;

0x4b58
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 1657 24
##############     xcb_ret.sequence = |x|cb_send_request(c, XCB_REQUEST_CHECKED, xcb_parts + 2, &xcb_req);

0x4b58:	mov esi, 1                           #    xcb_ret.sequence = |x|cb_send_request(c, XCB_REQUEST_CHECKED, xcb_parts + 2, &xcb_req);

0x4b5d
		operands[0].type: MEM
			operands[0].mem.base: REG = rsp
			operands[0].mem.disp: 0xc
0xc 12
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 1650 23
##############     xcb_out.attribute |=| attribute;

0x4b5d:	mov dword ptr [rsp + 0xc], edx       #    xcb_out.attribute |=| attribute;

0x4b61
		operands[1].type: MEM
			operands[1].mem.base: REG = rsp
			operands[1].mem.disp: 0x30
0x30 48
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 1657 24
##############     x

##############     xcb_out.width |=| width;

0x4fea:	mov word ptr [rsp + 0xc], cx         #    xcb_out.width |=| width;

0x4fef
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0x398a
0x398a 14730
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 2025 24
##############     xcb_ret.sequence = |x|cb_send_request(c, 0, xcb_parts + 2, &xcb_req);

0x4fef:	lea rcx, [rip + 0x398a]              #    xcb_ret.sequence = |x|cb_send_request(c, 0, xcb_parts + 2, &xcb_req);

0x4ff6
		operands[0].type: MEM
			operands[0].mem.base: REG = rsp
			operands[0].mem.disp: 0xe
0xe 14
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 2018 20
##############     xcb_out.height |=| height;

0x4ff6:	mov word ptr [rsp + 0xe], r8w        #    xcb_out.height |=| height;

0x4ffc
		operands[0].type: MEM
			operands[0].mem.base: REG = rsp
			operands[0].mem.disp: 0x30
0x30 48
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 2020 27
####

##############     xcb_out.drw_y |=| drw_y;

0x52c3:	mov word ptr [rsp + 0x1e], ax        #    xcb_out.drw_y |=| drw_y;

0x52c8
		operands[1].type: MEM
			operands[1].mem.base: REG = rsp
			operands[1].mem.disp: 0xd8
0xd8 216
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 2213 19
##############     xcb_out.drw_w |=| drw_w;

0x52c8:	mov eax, dword ptr [rsp + 0xd8]      #    xcb_out.drw_w |=| drw_w;

0x52cf
		operands[0].type: MEM
			operands[0].mem.base: REG = rsp
			operands[0].mem.disp: 0x60
0x60 96
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 2220 27
##############     xcb_parts[3].iov_base |=| 0;

0x52cf:	mov qword ptr [rsp + 0x60], 0        #    xcb_parts[3].iov_base |=| 0;

0x52d8
		operands[0].type: MEM
			operands[0].mem.base: REG = rsp
			operands[0].mem.disp: 0x20
0x20 32
/home/nahid/temp_dir/libxcb-shm0:amd64/libxcb-1.14/build/src/xv.c 2213 19
##############     xcb_out.drw_w |=| drw_w;

0x52d8:	mov word ptr [rsp + 0x20], ax        #   

In [18]:
print("SUBHANALLAH")

SUBHANALLAH
